## <3-2 트랜스포머 살펴보기>

트랜스포머는 2017년 구글이 제안한 ***시퀀스-투-시퀀스(sequence-to-sequence)*** 모델입니다. 최근 자연어 처리에서는 BERT나 GPT같은 트랜스포머 기반 언어 모델이 각광받고 있습니다. 그 성능이 좋기 때문인데요, 왜 성능이 좋은지, 핵심 동작 원리는 무엇인지 살펴보겠습니다.

### 시퀀스-투-시퀀스  
---
트랜스포머란 기계 번역 등 시퀀스-투-시퀀스 과제를 수행하는 모델입니다. 여기에서 시퀀스란 단어 같은 무언가의 나열을 의미하는데요, 시퀀스-투-시퀀스는 특정 속성을 지닌 시퀀스를 다른 속성의 시퀀스로 변환하는 작업을 가리킵니다.  

기계 번역을 예시로 시퀀스-투-시퀀스가 어떤 task인지 알아봅시다. 기계 번역이란 어떤 언어(source language)의 토큰 시퀀스를 다른 언어(target language)의 토큰 시퀀스로 변환하는 과제입니다.  
예를 들면 다음과 같습니다.

<center>어제, 카페, 갔었어, 거기, 사람, 많더라   (소스 언어)</center>  

<center>⬇</center>  

<center>I,went, to, the, cafe, there, were, many, people, there   (타깃 언어)</center>

자세히 살펴보면 소스 시퀀스의 길이(토큰 6개)와 타깃 시퀀스의 길이(10개)가 다르다는 점을 알 수 있습니다.  
이처럼 시퀀스-투-시퀀스 task는 소스와 타깃의 길이가 달라도 해당 과제를 수행하는 데 문제가 없어야 합니다.

### 인코더와 디코더
---
트랜스포머는 시퀀스-투-시퀀스 과제 수행에 특화된 모델입니다. 임의의 시퀀스를 해당 시퀀스와 속성이 다른 시퀀스로 변환하는 작업이라면 꼭 기계 번역이 아니더라도 수행할 수 있습니다.  예를 들어 필리핀 앞바다의 한 달 치 기온 데이터를 가지고 앞으로 1주일간 하루 단위로 태풍이 발생할지를 맞히는 과제(기온의 시퀀스 ➡ 태풍 발생 여부의 시퀀스) 역시 트랜스포머가 할 수 있는 일입니다.  

시퀀스-투-시퀀스 과제를 수행하는 모델은 다음 그림처럼 대개 ***인코더(encoder)*** 와 ***디코더(decoder)*** 2개 파트로 구성됩니다.

<center><그림 3-7 인코더, 디코더></center>  


<center>인코더(소스 시퀀스 압축) ➡ 소스 정보 ➡ 디코더(타깃 시퀀스 생성)</center>  

인코더는 소스 시퀀스의 정보를 압축해 디코더로 보내는 역할을 담당합니다. 인코더가 소스 시퀀스 정보를 압축하는 과정을 ***인코딩*** 이라고 합니다. 그리고 디코더는 인코더가 보내 준 소스 시퀀스 정보를 받아서 타깃 시퀀스를 생성합니다. 그리고 디코더는 인코더가 보내 준 소스 시퀀스 정보를 받아서 타깃 시퀀스를 생성하는 과정을 ***디코딩*** 이라고 합니다. 예를 들어 기계 번역에서는 인코더가 한국어 문장을 압축해 디코더에 보내고, 디코더는 이를 받아 영어로 번역합니다.

트랜스포머 역시 인코더와 디코더 구조를 따르며 이를 그림으로 나타내면 다음과 같습니다.  
인코더의 입력은 소스 시퀀스이고 디코더의 입력은 타깃 시퀀스의 일부입니다. 3장에서는 이 그림을 세분해서 트랜스포머의 동작 원리를 자세하게 살펴봅니다.

<그림 3-8 트랜스포머의 구조>

![](https://i.imgur.com/Rk5wkBQ.png)  



이미지 출처 : ratsgo's NLPBOOK

### 모델 학습과 인퍼런스
---
그림 3-8을 바탕으로 트랜스포머가 어떻게 학습하는지 살펴보겠습니다. 이번 학습은 그림 3-9처럼 I를 맞혀야 하는 차례라고 가정해 봅시다.

<center><그림 3-9 'i'를 맞히는 학습></center>

<center>어제 카페 갔었어, 거기 사람 많더라 ➡ 인코더 ➡ 디코더</center>  


<center> (S) ➡ 디코더 ➡ I</center>


cf) 따로 그렸지만, 인코더&디코더 둘이 연결되어 있는 형태입니다!

트랜스포머의 최종 출력, 즉 디코더 출력(그림 3-8에서 다음 토큰 확률)은 타깃 언어의 어휘수만큼의 차원으로 구성된 ***벡터(vector)*** 입니다. 이 벡터의 특징은 요솟(element)값이 모두 확률이라는 점입니다. 예를 들어 타깃 언어의 어휘가 총 3만 개라고 가정해 보면 디코더 출력은 3만 차원의 벡터입니다. 이 벡터의 요솟값 3만 개 각각은 확률이므로 0 이상 1 이하의 값을 가지며 모두 더하면 1이 됩니다.  

트랜스포머의 학습은 인코더와 디코더 입력이 주어졌을 때 정답에 해당하는 단어의 확률값을 높이는 방식으로 수행됩니다. 이를 나타낸 다음 그림에서 모델은 이번 시점의 정답인 I에 해당하는 확률은 높이고 나머지 단어의 확률은 낮아지도록 모델 전체를 갱신합니다.  


<center><그림 3-10 'i' 확률 높이기></center>  

<center>... cafe ⬇ ... I ⬆ ... to ⬇ ... the ⬇ ... went ⬇ ... were ⬇ ...</center>

이번에는 타깃 시퀀스 가운데 to를 맞힐 차례입니다. 다음 그림처럼 인코더 입력은 소스 시퀀스 전체입니다. 학습 과정 중 디코더 입력은 정답인 **\<S> I went**, 인퍼런스할 때 디코더 입력은  직전 디코딩 결과입니다.

<center><그림 3-13 'to'를 맞히는 학습></center>

<center>어제 카페 갔었어, 거기 사람 많더라 ➡ 인코더 ➡ 디코더</center>  


<center> (S) I went ➡ 디코더 ➡ to</center>


cf) 따로 그렸지만, 인코더&디코더 둘이 연결되어 있는 형태입니다!

학습 과정 중 인코더, 디코더 입력이 그림 3-13과 같은 상황에서 모델은 이번 시점의 정답인 to에 해당하는 확률은 높이고 나머지 단어의 확률은 낮아지도록 모델 전체를 갱신합니다.

<center><그림 3-14 'to' 확률 높이기></center>  

<center>... cafe ⬇ ... I ⬇ ... to ⬆ ... the ⬇ ... went ⬇ ... were ⬇ ...</center>

이러한 방식으로 말뭉치 전체를 반복해서 학습하면 한국어-영어 기계 번역을 성공적으로 수행할 수 있습니다.

### 트랜스포머 블록
---
다음 그림은 트랜스포머의 인코더 가운데 반복되는 요소를 떼어내 다시 나타낸 것입니다.  
이런 구조를 ***블록(block)*** 또는 ***레이어(layer)*** 라고 합니다. 트랜스포머의 인코더는 이러한 블록 수십 개를 쌓아서 구성합니다.

<그림 3-15 트랜스포머 인코더 블록>

![](https://i.imgur.com/NSmVlit.png) 

이미지 출처 : ratsgo's NLPBOOK

그림에서 확인할 수 있듯이 인코더 블록은 ***멀티 헤드 어텐션(multi-head attention)***, ***피드포워드 뉴럴 네트워크(feedforward neural network)***, ***잔차 연결(residual connection)*** 및 ***레이어 정규화(layer normalization)*** 등 3가지 요소로 구성돼 있습니다. 각 요소는 앞으로 자세히 다루겠습니다.  

디코더 쪽 블록의 구조도 인코더 블록과 본질적으로는 다르지 않습니다. 다만 ***마스크를 적용한 멀티 헤드 어텐션(masked multi-head attention)*** 이 인코더 쪽과 다르고, 인코더가 보내 온 정보와 디코더 입력을 함께 이용해 멀티 헤드 어텐션을 수행하는 모듈이 추가됐습니다.

<그림 3-16 트랜스포머 디코더 블록>

![](https://i.imgur.com/jmNALxv.png) 

이미지 출처 : ratsgo's NLPBOOK